<a href="https://colab.research.google.com/github/JuanHermann/BT_IFR2/blob/main/BT_IFR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from tabulate import tabulate

In [ ]:
#Funcao para calcular o IFR2 utilizando a média movel de 3 para o resultado do IFR não ficar tão agressivo por ser tão curto

def calcularIFR2(data, rsi_period=2, smooth_period=3):

    if "Close" not in data.columns or len(data) < smooth_period:
        print(f"Erro: Dados insuficientes. Necessário pelo menos {smooth_period} períodos.")
        return data

    # 2. Variação de Preço
    delta = data["Close"].diff(periods=1)

    # 3. Ganhos (Up) e Perdas (Down) - Perda como valor positivo
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # 4. Suavização EMA (Wilder's Smoothing)
    # O método 'ewm' com adjust=False e span=smooth_period é o equivalente no Pandas
    # à suavização de Welles Wilder (RMA).

    # Usamos min_periods=smooth_period para o cálculo começar no ponto correto.
    avg_gain = gain.ewm(span=smooth_period, min_periods=smooth_period, adjust=False).mean()
    avg_loss = loss.ewm(span=smooth_period, min_periods=smooth_period, adjust=False).mean()

    # 5. Cálculo do RS e RSI
    epsilon = 1e-10 # Adiciona epsilon para estabilidade numérica e evitar divisão por zero

    # Força Relativa (RS)
    rs = avg_gain / (avg_loss + epsilon)

    # Índice de Força Relativa (RSI)
    rsi = 100 - (100 / (1 + rs))

    # 6. Tratamento de Casos Extremos (Divisão por zero)
    # Garante que o RSI = 100 quando não há perda (avg_loss = 0) e há ganho.
    loss_is_zero = np.isclose(avg_loss, 0, atol=epsilon)
    gain_is_positive = (avg_gain > 0)

    rsi[loss_is_zero & gain_is_positive] = 100

    # 7. Renomeia e Retorna
    # Nome da coluna ajustado para refletir o período de suavização real
    data["RSI_2"] = rsi

    # Retorna o DataFrame a partir do ponto onde o RSI é calculado
    return data.iloc[smooth_period - 1:]

In [ ]:
def finalizaTrade(trades,entry_date, exit_date, entry_price,exit_price,max_holding_days):

  profit = exit_price - entry_price
  trades.append({
                'Entry_Date': entry_date,
                'Exit_Date': exit_date,
                'Entry_Price': entry_price,
                'Exit_Price': exit_price,
                'P/L': profit,
                'Result': 'Win' if profit > 0 else 'Loss',
                'Max_Hold_Days': max_holding_days

            })

In [ ]:
def gerar_tabela(list_of_trades_data):

    cabecalho_multi_nivel = []
    periodos = ["5 ANOS", "3 ANOS", "1 ANO", "6 MESES"]
    metricas = ["QTDE", "F. LUCRO", "%"]

    for periodo in periodos:
        for metrica in metricas:
            cabecalho_multi_nivel.append((periodo, metrica))

    cabecalho_multi_nivel.append(("CANDLES", ""))
    cabecalho_multi_nivel.append(("DROPDOWN MAX", ""))

    colunas = pd.MultiIndex.from_tuples(cabecalho_multi_nivel, names=['Período', 'Métrica'])

    # --- 2. Processar cada conjunto de trades na lista ---
    resultados = []
    for trades_data in list_of_trades_data:
        # Convert the trades data to a DataFrame if it's not already
        trades_df = pd.DataFrame(trades_data)

        if trades_df.empty:
            # Append a row of zeros or NaNs if there are no trades for this set
            resultados.append([0] * len(colunas))
            continue

        # Ensure 'Entry_Date' is datetime for filtering
        trades_df['Entry_Date'] = pd.to_datetime(trades_df['Entry_Date'])

        # Filter trades based on time periods
        trades_df_5_years  = trades_df[trades_df['Entry_Date'] >= (date.today() - pd.DateOffset(years=5))]
        trades_df_3_years  = trades_df[trades_df['Entry_Date'] >= (date.today() - pd.DateOffset(years=3))]
        trades_df_1_year   = trades_df[trades_df['Entry_Date'] >= (date.today() - pd.DateOffset(years=1))]
        trades_df_6_months = trades_df[trades_df['Entry_Date'] >= (date.today() - pd.DateOffset(months=6))]

        # Calculate metrics for each period
        row_data = [
            # 5 ANOS (QTDE, F. LUCRO, %)
            len(trades_df_5_years), getProfitFactor(trades_df_5_years), getWinRate(trades_df_5_years),
            # 3 ANOS (QTDE, F. LUCRO, %)
            len(trades_df_3_years), getProfitFactor(trades_df_3_years), getWinRate(trades_df_3_years),
            # 1 ANO (QTDE, F. LUCRO, %)
            len(trades_df_1_year), getProfitFactor(trades_df_1_year), getWinRate(trades_df_1_year),
            # 6 MESES (QTDE, F. LUCRO, %)
            len(trades_df_6_months), getProfitFactor(trades_df_6_months), getWinRate(trades_df_6_months),
            # CANDLES (Assuming you want an aggregate like mean)
            trades_df['Max_Hold_Days'][0],
            # DROP MAX
            getMaxDropdown(trades_df)
        ]
        resultados.append(row_data)


    # --- 3. CRIAR O DATAFRAME ---
    df = pd.DataFrame(resultados, columns=colunas)
    df.index.name = None # Opcional: Remover o nome do índice da primeira linha

    return df
#gerar_tabela(trades_list_of_lists)

In [ ]:
def getWinRate(trades_df):
    winning_trades = len(trades_df[trades_df['P/L'] > 0])
    win_rate = (winning_trades / len(trades_df)) * 100
    return f"{win_rate:.2f}%"

In [ ]:
def getProfitFactor(trades_df):
    total_gross_profit = trades_df[trades_df['P/L'] > 0]['P/L'].sum()
    total_gross_loss = abs(trades_df[trades_df['P/L'] < 0]['P/L'].sum())
    profit_factor = total_gross_profit / total_gross_loss if total_gross_loss > 0 else np.inf
    return f"{profit_factor:.2f}"

In [ ]:
def getMaxDropdown(trades_df):
    # 6.3. Drawdown Máximo (Max Drawdown)
    # Calcula o retorno cumulativo (equity curve)
    trades_df['Cumulative_P/L'] = trades_df['P/L'].cumsum()
    equity_curve = trades_df['Cumulative_P/L']
    # Calcula o máximo valor da curva de patrimônio até o momento
    peak = equity_curve.expanding().max()
    # Calcula o Drawdown (do pico até o valor atual)
    drawdown = (equity_curve - peak) / peak.mask(peak == 0, 1)
    max_drawdown = abs(drawdown.min()) * 100
    return f"{max_drawdown:.2f}%"

In [ ]:
def getTableAllResults(all_tickers_results):

  # Create an empty list to store the data for the new DataFrame
  data_for_new_df = []

  # Iterate through the results for each ticker
  for ticker, result_df in all_tickers_results.items():
    # Check if the result is a DataFrame and not empty
    if isinstance(result_df, pd.DataFrame) and not result_df.empty:
      # Get the first row of the results DataFrame
      best_row = selecionar_melhor_parametro_ponderado(result_df)
      # Add the ticker to the dictionary for the first row
      best_row['Ticker'] = ticker[:-3]
      # Append the data to the list
      data_for_new_df.append(best_row)
    else:
      # Handle cases where there are no results or an error occurred
      print(f"Could not get results for {ticker}: {result_df}")


  # Create a new DataFrame from the collected data
  new_df = pd.DataFrame(data_for_new_df)

  # Reorder columns to have Ticker as the first column
  if 'Ticker' in new_df.columns:
    cols = ['Ticker'] + [col for col in new_df.columns if col != 'Ticker']
    new_df = new_df[cols]

  return new_df

In [ ]:
def selecionar_melhor_parametro_ponderado(df_resultados: pd.DataFrame) -> pd.DataFrame:

    # 1. Preparação: Limpeza, normalização e cálculo do Score para CADA PERÍODO

    # Ensure index name is set for the score DataFrame
    df_scores = df_resultados.index.to_frame(name='CANDLES').copy()


    for periodo, peso in PERIODOS_E_PESOS.items():

        # Extrai as métricas
        qtde = df_resultados[(periodo, 'QTDE')]

        # Convert 'F. LUCRO' to numeric, coercing errors
        f_lucro = pd.to_numeric(df_resultados[(periodo, 'F. LUCRO')], errors='coerce')

        # Convert '%' to numeric (float between 0 and 1), coercing errors
        taxa_acerto_str = df_resultados[(periodo, '%')].astype(str).str.replace('%', '').str.replace(',', '.')
        taxa_acerto = pd.to_numeric(taxa_acerto_str, errors='coerce') / 100

        # Create a column for the Raw Score of this period
        coluna_score_bruto = f'SCORE_BRUTO_{periodo.replace(" ", "_")}'

        # Formula for Raw Score (Quality)
        # Score = (Win Rate * Profit Factor) * sqrt(Quantity of Trades)
        # Handle potential NaN values from coercion by filling with 0 for score calculation
        df_scores[coluna_score_bruto] = (taxa_acerto.fillna(0) * f_lucro.fillna(0) * np.sqrt(qtde.fillna(0)))

        # Apply the Weight to the Final Score
        df_scores[coluna_score_bruto] *= peso

    # 2. Calculation of FINAL SCORE (Weighted Sum)

    score_cols = [col for col in df_scores.columns if col.startswith('SCORE_BRUTO')]
    df_scores['SCORE_FINAL_PONDERADO'] = df_scores[score_cols].sum(axis=1)

    # 3. Selection of the Best Result

    melhor_parametro_index = df_scores['SCORE_FINAL_PONDERADO'].idxmax()

    # Return the final result, including the complete row from the original DF and the Final Score
    melhor_linha = df_resultados.loc[melhor_parametro_index].to_frame().T
    # Add the SCORE FINAL PONDERADO to the output DataFrame
    melhor_linha[('SCORE', 'FINAL')] = df_scores.loc[melhor_parametro_index, 'SCORE_FINAL_PONDERADO']

    return  df_resultados.iloc[melhor_parametro_index].to_dict()

In [ ]:
def showAllResults(all_tickers_results):
# --- Display results for all tickers ---
  print("\n--- Backtest Results for All Tickers ---")
  for ticker, result in all_tickers_results.items():
      print(f"\nResults for {ticker}:")
      if isinstance(result, pd.DataFrame):
          display(result)
      else:
          print(result)

In [ ]:
def downloadAcoes(TICKERS, START_DATE =(date.today() - pd.DateOffset(years=5)), DIAS_ANTERIORES = 30):
  start_date_download = START_DATE - pd.DateOffset(days=DIAS_ANTERIORES)
  print(f"realizando Backtest entre as datas {START_DATE.strftime("%d/%m/%Y")} e {date.today().strftime("%Y-%m-%d")}")

  try:
      print(f"Baixando dados para {TICKERS} desde {start_date_download}...")
      data = yf.download(TICKERS, start=start_date_download,  auto_adjust=True)

      if data.empty:
          raise ValueError(f"Não foi possível baixar dados para os tickers {TICKERS}. Verifique os símbolos.")
      return data

  except Exception as e:
      print(f"ERRO: Não foi possível obter dados para os tickers {TICKERS}. Verifique sua conexão ou os tickers. Detalhe: {e}")

In [ ]:
def validaSaida(df,i,trades,entry_date,current_day,entry_price, maxHoldDay):
  exit_target= max(df.iloc[i-1]['High'].item(),df.iloc[i-2]['High'].item())
  #Vender na maxima dos ultimos 2 candles
  if current_day['High'].item() >= exit_target:
    #Valida se não abriu em um gap acima do valor de maxima, caso abriu, vende no preco de abertura
    if current_day['Open'].item() >= exit_target:
      finalizaTrade(trades,entry_date,current_day.name,entry_price,current_day['Open'].item(), maxHoldDay)
    else:
      finalizaTrade(trades,entry_date,current_day.name,entry_price,exit_target, maxHoldDay)
    return True
  return False

In [ ]:
def calcularTrades(df):
   trades_list_of_lists = []
   for maxHoldDay in MAX_HOLDING_DAYS_RANGE:
       trades = []
       in_trade = False
       entry_price = 0
       entry_date = None
       exit_target = 0
       holding_days = 0
       for i in range(len(df)):
          current_day = df.iloc[i]
          if current_day.name < START_DATE:
               continue
          if in_trade:
               holding_days += 1
               #Vender na abertura do dia, stop no tempo
               if holding_days >= maxHoldDay:
                   finalizaTrade(trades,entry_date,current_day.name,entry_price,current_day['Open'].item(), maxHoldDay)
                   in_trade = False
                   continue
               if validaSaida(df,i,trades,entry_date,current_day,entry_price, maxHoldDay):
                   in_trade = False
                   continue
          else:
              #Compras caso a ação abre em um gap de baixa
              if df.iloc[i-1]['RSI_2'].item() < RSI_THRESHOLD:
                if COMPRAR_ABERTURA_EM_GAP:
                  if current_day['Open'].item() < df.iloc[i-1]['Close'].item():
                   in_trade = True
                   entry_price = current_day['Open'].item()
                   entry_date = current_day.name
                   holding_days = 0
                   #Valida a possibilidade de saida no mesmo candle de entrada
                   if validaSaida(df,i,trades,entry_date,current_day,entry_price, maxHoldDay):
                    in_trade = False
                   continue
                if current_day['Low'].item() <= df.iloc[i-1]['Close'].item() <= current_day['High'].item():
                   in_trade = True
                   entry_price = df.iloc[i-1]['Close'].item()
                   entry_date = current_day.name
                   holding_days = 0
                   #Valida a possibilidade de saida no mesmo candle de entrada
                   if validaSaida(df,i,trades,entry_date,current_day,entry_price, maxHoldDay):
                    in_trade = False
       #Ignora ultimo trade caso não tenha sido finalizado
       if in_trade:
           pass
       if trades:
           trades_list_of_lists.append(trades)
   return trades_list_of_lists


In [ ]:
def displayVirgula(df_display):
  df_display = df_display.sort_values(by=('3 ANOS', 'F. LUCRO'), ascending=False)

  # Iterate through multi-index columns and apply formatting for display
  for col_header in df_display.columns:
      # Check if the metric part of the multi-index is 'F. LUCRO', '%' or 'DROPDOWN MAX'
      if col_header[1] in ['F. LUCRO', '%', 'DROPDOWN MAX']:
          # Ensure the column is of string type before applying .str.replace
          df_display[col_header] = df_display[col_header].astype(str).str.replace('.', ',', regex=False)

  # Display the formatted DataFrame
  display(df_display)

In [ ]:
# -------------ANOTACOES -----------
# A QUANTIDADE DE TRADES PODE SEM 1 NUMERO MENOR QUE O PROFIT POIS O PROFIT CONTA OS TRADES NAO FINALIZADOS.
# O numero de candles significa por exemplo, candles 6 Candles significa que eu vou compra no dia 1 e vou ficar 6 candles completos contando com oq eu comprei( pq comprei no comeco) e na abertura do candle 7 eu vendo

# -------------- TO DO -------------
# ALTERAR MAX DROPDOWN COM VALOR DE INVESTIMENTO PARA O RESULTADO FICAR MAIS REAL
# VALIDAR ALGORITIMO QUE DECIDE A MELHOR LINHA
# QTDE DE TRADES DIFERENTE POIS AQUI OS VALORES NÃO ESTAO ARREDONDADOS (FFECHAMENTO DE ONTEM 15,8988 E O MINIMO DE HOJE FOI 15,8989. ENTAO NÃO ACONTECE O TRADE)

TICKERS = ["ODPV3.SA"]#,"CPFE3.SA","MOTV3.SA","COGN3.SA","IRBR3.SA","ELET6.SA","VIVT3.SA","MULT3.SA","SBSP3.SA","EQTL3.SA","ITUB4.SA","CPLE6.SA",
    #       "GRND3.SA","EGIE3.SA","TOTS3.SA","TIMS3.SA","ABEV3.SA","PRIO3.SA","PETR4.SA","RAIL3.SA","PSSA3.SA","WEGE3.SA","GGBR4.SA","ENEV3.SA",
     #      "ISAE4.SA","TAEE11.SA","CMIG4.SA","EZTC3.SA","HYPE3.SA","TGMA3.SA","RENT3.SA","SANB11.SA","BOVA11.SA","SLCE3.SA","EMBR3.SA","SUZB3.SA"]

PERIODOS_E_PESOS = {
        "5 ANOS": 1.0,
        "3 ANOS": 0.8,
        "1 ANO": 0.6,
        "6 MESES": 0.4,
    }
START_DATE = date.today() - pd.DateOffset(years=5)
RSI_PERIOD = 2
RSI_THRESHOLD = 25
MAX_HOLDING_DAYS_RANGE = range(3, 9)
COMPRAR_ABERTURA_EM_GAP = False

all_tickers_results = {}

data = downloadAcoes(TICKERS)

for ticker in TICKERS:
    print(f"--- Processing Ticker: {ticker} ---")

    # Select the data for the current ticker from the multi-indexed DataFrame
    # Check if data was successfully downloaded and the ticker exists in the downloaded data
    if 'data' in locals() and not data.empty and ticker in data.columns.get_level_values(1):
        # Select all rows (:) and columns where the second level of the MultiIndex (ticker) matches the current ticker
        df = data.xs(ticker, level='Ticker', axis=1).copy()

        df = calcularIFR2(df)

        # Remove linhas com valores nulos
        df.dropna(inplace=True)

        lista_trades = calcularTrades(df)

        if lista_trades:
            all_tickers_results[ticker] = gerar_tabela(lista_trades)
        else:
            all_tickers_results[ticker] = "No trades found"
    else:
        # If data download failed for this specific ticker or ticker not found in downloaded data, skip processing
        if ticker not in all_tickers_results: # Avoid overwriting existing error message if download failed
             all_tickers_results[ticker] = "Ticker not found in downloaded data or data download failed"
        print(f"Skipping processing for {ticker} due to data issue.")

sorted_df = getTableAllResults(all_tickers_results)
displayVirgula(sorted_df)

In [ ]:
#Metodo para validar os trades feitos pelo codigo
def filtrarTradesHoldingdays(ticker, data_inicio=(date.today() - pd.DateOffset(months=6)), max_hold_days=None):
  df = data.xs(ticker, level='Ticker', axis=1).copy()
  df = calcularIFR2(df)
  df.dropna(inplace=True)

  lista_trades = calcularTrades(df)
  filtered_trades_by_date = []

  for trades_for_max_hold_days in lista_trades:
      # Convert the list of dictionaries to a DataFrame
      trades_df = pd.DataFrame(trades_for_max_hold_days)

      # Ensure 'Entry_Date' is in datetime format
      trades_df['Entry_Date'] = pd.to_datetime(trades_df['Entry_Date'])

      # Filter the DataFrame for the specified Entry_Date
      filtered_df = trades_df[trades_df['Entry_Date'] >= data_inicio]

      # Further filter by max_hold_days if provided
      if max_hold_days is not None:
          filtered_df = filtered_df[filtered_df['Max_Hold_Days'] == max_hold_days]

      if not filtered_df.empty:
          filtered_trades_by_date.append(filtered_df)

  if filtered_trades_by_date:
      print(f"Trades with Entry_Date = {data_inicio} and Max_Hold_Days = {max_hold_days}:")
      for i, df_result in enumerate(filtered_trades_by_date):
          print(f"\nResults for Max_Hold_Days combination {i+1}:")
          display(df_result)
  else:
      print(f"No trades found with Entry_Date = {data_inicio} and Max_Hold_Days = {max_hold_days}.")

#filtrarTradesHoldingdays('CMIG4.SA',(date.today() - pd.DateOffset(years=1)),5)